In [2]:
from pathlib import Path

import dask
import iris

In [3]:
basedir = Path('/gws/nopw/j04/kscale/DYAMOND3_example_data/sample_data_hirerarchy/5km-RAL3/glm/field.pp')

In [4]:
streams = sorted(basedir.glob('apver*.pp'))
streams

[PosixPath('/gws/nopw/j04/kscale/DYAMOND3_example_data/sample_data_hirerarchy/5km-RAL3/glm/field.pp/apvera.pp'),
 PosixPath('/gws/nopw/j04/kscale/DYAMOND3_example_data/sample_data_hirerarchy/5km-RAL3/glm/field.pp/apverb.pp'),
 PosixPath('/gws/nopw/j04/kscale/DYAMOND3_example_data/sample_data_hirerarchy/5km-RAL3/glm/field.pp/apverc.pp'),
 PosixPath('/gws/nopw/j04/kscale/DYAMOND3_example_data/sample_data_hirerarchy/5km-RAL3/glm/field.pp/apverd.pp')]

In [5]:
streams

[PosixPath('/gws/nopw/j04/kscale/DYAMOND3_example_data/sample_data_hirerarchy/5km-RAL3/glm/field.pp/apvera.pp'),
 PosixPath('/gws/nopw/j04/kscale/DYAMOND3_example_data/sample_data_hirerarchy/5km-RAL3/glm/field.pp/apverb.pp'),
 PosixPath('/gws/nopw/j04/kscale/DYAMOND3_example_data/sample_data_hirerarchy/5km-RAL3/glm/field.pp/apverc.pp'),
 PosixPath('/gws/nopw/j04/kscale/DYAMOND3_example_data/sample_data_hirerarchy/5km-RAL3/glm/field.pp/apverd.pp')]

In [6]:
paths = [sorted(s.glob('*.pp'))[0] for s in streams]

In [7]:
paths

[PosixPath('/gws/nopw/j04/kscale/DYAMOND3_example_data/sample_data_hirerarchy/5km-RAL3/glm/field.pp/apvera.pp/glm.n2560_RAL3p3.apvera_20200120T00.pp'),
 PosixPath('/gws/nopw/j04/kscale/DYAMOND3_example_data/sample_data_hirerarchy/5km-RAL3/glm/field.pp/apverb.pp/glm.n2560_RAL3p3.apverb_20200120T00.pp'),
 PosixPath('/gws/nopw/j04/kscale/DYAMOND3_example_data/sample_data_hirerarchy/5km-RAL3/glm/field.pp/apverc.pp/glm.n2560_RAL3p3.apverc_20200120T00.pp'),
 PosixPath('/gws/nopw/j04/kscale/DYAMOND3_example_data/sample_data_hirerarchy/5km-RAL3/glm/field.pp/apverd.pp/glm.n2560_RAL3p3.apverd_20200120T00.pp')]

In [8]:
paths[:2]

[PosixPath('/gws/nopw/j04/kscale/DYAMOND3_example_data/sample_data_hirerarchy/5km-RAL3/glm/field.pp/apvera.pp/glm.n2560_RAL3p3.apvera_20200120T00.pp'),
 PosixPath('/gws/nopw/j04/kscale/DYAMOND3_example_data/sample_data_hirerarchy/5km-RAL3/glm/field.pp/apverb.pp/glm.n2560_RAL3p3.apverb_20200120T00.pp')]

In [9]:
cubes = iris.load(paths[:2])

/home/users/mmuetz/miniforge3/envs/hackathon_env/lib/python3.12/site-packages/pyproj/network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)


In [10]:
@dask.delayed
def cube_sum(cube):
    return cube.data.sum()

In [11]:
res = [cube_sum(cube) for cube in cubes[:5]]


In [12]:
res

[Delayed('cube_sum-1c9e5ec6-9142-415e-816d-3611a3ed9ce8'),
 Delayed('cube_sum-1022d2d0-9134-4bd9-913b-e6917f7c7604'),
 Delayed('cube_sum-da646cfb-0f43-4cbc-99f4-ec7d3515e0b4'),
 Delayed('cube_sum-8df24387-ed40-4b65-8ebb-a0acc4e826f1'),
 Delayed('cube_sum-f81a3ee6-ef49-4c63-8119-48a05d3f1bfe')]

In [18]:
dask.compute(res)

([np.float32(29897791000.0),
  np.float32(3338188500.0),
  np.float32(2519572000000.0),
  np.float32(4763554000.0),
  np.float32(8364943400.0)],)

In [16]:
cubes2 = iris.load(paths[:2])

In [19]:
res2 = [c.data.sum() for c in cubes2[:5]]

In [15]:
res2

[np.float32(29897791000.0),
 np.float32(3338188500.0),
 np.float32(2519572000000.0),
 np.float32(4763554000.0),
 np.float32(8364943400.0)]

In [28]:
import numpy as np
import xarray as xr

In [20]:
def model_level_to_pressure(cube):
    return cube

In [21]:
def cube_to_da(cube):
    return xr.DataArray.from_iris(cube)

In [36]:
def da_to_healpix(da):
    rdims = list(da.dims[:-2])
    coords = {d: da[d] for d in rdims}
    shape = list(da.shape[:-2])
    ncell = 12 * 4**5
    ndata = np.zeros(shape + [ncell])
    coords['cell'] = np.arange(ncell)
    return xr.DataArray(ndata, dims=rdims + ['cell'], coords=coords, name=da.name, attrs=da.attrs)

In [33]:
def write_to_zarr(da_hp):
    print(da_hp)
    return da_hp

In [38]:
res = []
for cube in cubes[:5]:
    print(cube.name())
    if cube == cubes[0]:
        pcube = dask.delayed(model_level_to_pressure)(cube)
    else:
        pcube = cube
    da = dask.delayed(cube_to_da)(pcube)
    da_hp = dask.delayed(da_to_healpix)(da)
    res.append(dask.delayed(write_to_zarr)(da_hp))
res

m01s01i202
m01s09i223
m01s30i403
m01s30i461
m01s30i462


[Delayed('write_to_zarr-393e1c63-81a2-4553-bb92-ab57c8df9366'),
 Delayed('write_to_zarr-8a3cb3d8-d183-45c5-aff8-c441a64badfe'),
 Delayed('write_to_zarr-a7fb68dc-45bf-4264-a260-704bc5b8619d'),
 Delayed('write_to_zarr-4f8e2282-126d-40a6-ab10-171118c70fdc'),
 Delayed('write_to_zarr-eaee1a77-363d-47b3-a17b-4f7b09b1edfa')]

In [39]:
dask.compute(res)

([<xarray.DataArray (time: 13, cell: 12288)> Size: 1MB
  array([[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]])
  Coordinates:
    * time     (time) datetime64[ns] 104B 2020-01-20T00:02:00 ... 2020-01-20T12...
    * cell     (cell) int64 98kB 0 1 2 3 4 5 ... 12283 12284 12285 12286 12287
  Attributes:
      source:      Data from Met Office Unified Model
      um_version:  13.5
      STASH:       m01s01i202,
  <xarray.DataArray (time: 12, cell: 12288)> Size: 1MB
  array([[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]])
  Coordinates:
    * time     (time) datetime64[ns] 96B 2020-01-20T01:00:00 ... 